In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append('../')

import src.models as models
import src.transform as transform
import src.video as video
import os
import logging
from tqdm import tqdm

In [2]:
# Init logging
logger = logging.getLogger('styleTransfer')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.FileHandler("my_log.log", mode='w'))

# Video Extraction

In [3]:
# Clean the stuff from the last run
video.clean_dir('../res/atla/')
n_frames, fps = video.extract_video('../res/atla.mp4', '../res/atla/', n_frames=25)

Successfully read and saved  26 frames


In [4]:
# Load Style Img
style_img_1 = transform.load_image('../res/style/starry_night.jpg')

# Load Style Img 2
style_img_2 = transform.load_image('../res/style/mona_lisa.jpg')

lamb=1.0

In [5]:
assert style_img_1.size() == style_img_2.size()

# Running Style Transfer

In [6]:
vgg19 = models.vgg19(weights=models.vgg_19_default_weights).features.eval()
mean = models.vgg19_normalization_mean
std = models.vgg19_normalization_std

In [7]:
count = 0
video.clean_dir('../res/output')
for img in tqdm(os.listdir('../res/atla')):
    if not img.endswith('.jpg'):
        continue

    lamb = 1 - count / 25

    content_img = transform.load_image(os.path.join('../res/atla', img))
    assert content_img.size() == style_img_1.size()

    output = transform.run_style_transfer(vgg19, 
                             mean,
                             std,
                             content_img,
                             style_img_1,
                             None,
                             lamb=lamb)
    output_img = transform.to_PIL(output)
    output_img.save(f'../res/output/frame{count:04d}.png')
    count +=1

c:\Users\Jackson Paull\.conda\envs\img_proc\lib\site-packages\torch\utils\_device.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


In [ ]:
# Greatest entry with frame < end_frame is the current step. frame
keypoints = [
    # End Frame, path to style image 1,   path to style image 2,             interpolation function
    (1000,      'path_to_img_1',        'path_2_img_2',        '<lambda func dependent upon frame number>'),
    (2000),
    (3000),
    (4000),
    (5000),
    (6000),
    (7000),
    (8000),
    (-1) # Until the end
]

# Video Encoding

In [8]:
video.encode_video('../res/output', '../output.mp4', fps=fps)

# Don't need to store the video in a lossy format 3 times
video.clean_dir('../res/atla')
video.clean_dir('../res/output')